In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
from scipy.stats import linregress
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import warnings 
%matplotlib inline

# 1. Datenpfad zuweisen

In [4]:
csv_1 = "data/kunden.csv"
csv_2 = "data/besucher.csv"

# 2. Laden der Dateien mit Pandas

In [5]:
dp_1 = pd.read_csv(csv_1, sep = ";")
dp_2 = pd.read_csv(csv_2, sep = ";")

In [6]:
dp_1

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1.0,43.0,K0310
1,54,77644.0,103145.70,1.0,40.0,K1042
2,55,44341.0,80565.16,0.0,37.0,K0382
3,49,67271.0,83949.89,0.0,42.0,K0498
4,46,49832.0,93781.58,0.0,41.0,K0552
...,...,...,...,...,...,...
1099,70,69581.0,103751.58,1.0,51.0,K0467
1100,38,48301.0,64230.98,1.0,33.0,K0122
1101,67,63547.0,103123.57,1.0,41.0,K1045
1102,37,32686.0,49215.87,1.0,36.0,K1096


In [7]:
dp_2

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,"39919,0953348874",0,"29,0",K1814
1,41,"53430,67308587748",0,"27,0",K1544
2,64,"73163,2169267002",0,"32,0",K1945
3,41,"36761,482581315344",0,"44,0",K1825
4,35,"42684,33163178136",1,"33,0",K1144
...,...,...,...,...,...
886,32,"34391,82496545645",1,"48,0",K1211
887,41,"68516,49543941053",0,"34,0",K1375
888,65,"89685,33344089528",0,"23,0",K1965
889,68,"95714,30526812703",0,"32,0",K1540


# 3. Überblick über Datentypen verschaffen

In [8]:
dp_1.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht    float64
Zeit          float64
KundeNr        object
dtype: object

In [9]:
dp_2.dtypes

Alter          int64
Einkommen     object
Geschlecht     int64
Zeit          object
KundeNr       object
dtype: object

# 4. Bestimmung fehlender Werte

Nur bei dp_1 Geschlecht, gibt es 5 fehlende Werte

In [10]:
dp_1.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    5
Zeit          0
KundeNr       0
dtype: int64

# 5. Nicht vorhandene Zaheln mit 1sen ersetzten

In [11]:
dp_1["Geschlecht"] = dp_1["Geschlecht"].fillna(1)
dp_1.isnull().sum()

Alter         0
Einkommen     0
Preis         0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

In [12]:
dp_2.isnull().sum()

Alter         0
Einkommen     0
Geschlecht    0
Zeit          0
KundeNr       0
dtype: int64

In [13]:
dp_1.describe().T

,count,mean,std,min,25%,50%,75%,max
Alter,1104.0,50.481884,1.222506e+01,30.000000,40.0000,51.000,61.000,1.200000e+02
Einkommen,1104.0,964474.817596,3.009470e+07,-62840.374523,46638.5000,59013.000,70298.000,1.000000e+09
Preis,1104.0,88975.198714,2.446407e+04,21471.650000,70609.4225,88931.455,107093.375,1.654823e+05
Geschlecht,1104.0,0.639493,4.803653e-01,0.000000,0.0000,1.000,1.000,1.000000e+00
Zeit,1104.0,44.375906,1.084804e+01,12.000000,37.0000,44.000,52.000,8.000000e+01


In [14]:
dp_2.describe().T

,count,mean,std,min,25%,50%,75%,max
Alter,891.0,49.881033,11.865889,30.0,40.0,49.0,60.0,70.0
Geschlecht,891.0,0.578002,0.494156,0.0,0.0,1.0,1.0,1.0


# 6. Übersicht verschaffen

dp_1: Hohes alter (120 Jahre) und sehr hohes und niedriges Einkommen
dp_2: Nichts außergewöhnliches

In [15]:
dp_1.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1.104000e+03,1104.000000,1104.000000,1104.000000
mean,50.481884,9.644748e+05,88975.198714,0.639493,44.375906
std,12.225059,3.009470e+07,24464.067781,0.480365,10.848040
min,30.000000,-6.284037e+04,21471.650000,0.000000,12.000000
25%,40.000000,4.663850e+04,70609.422500,0.000000,37.000000
50%,51.000000,5.901300e+04,88931.455000,1.000000,44.000000
75%,61.000000,7.029800e+04,107093.375000,1.000000,52.000000
max,120.000000,1.000000e+09,165482.310000,1.000000,80.000000


# 7. Alter überprüfen

Ausreißer sind 100 und 120 Jahre.
Modalwert ist 55 Jahre.
Ausreißer werden durch den Modalwert ersetzt.

In [16]:
dp_1["Alter"]. value_counts()

Alter
55     41
62     39
57     34
66     33
64     32
30     32
34     31
69     30
37     30
31     30
52     29
54     29
58     29
65     29
53     29
61     29
68     29
59     28
35     27
51     27
41     26
40     26
43     26
32     26
46     26
39     24
48     24
38     24
33     24
44     23
50     23
67     23
42     22
70     22
56     22
45     22
49     22
63     21
60     20
47     20
36     19
100     1
120     1
Name: count, dtype: int64

In [17]:
dp_1["Alter"] = dp_1["Alter"]. apply(lambda x: 55 if x >= 100 else x)
dp_1.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1.104000e+03,1104.000000,1104.000000,1104.000000
mean,50.382246,9.644748e+05,88975.198714,0.639493,44.375906
std,11.953017,3.009470e+07,24464.067781,0.480365,10.848040
min,30.000000,-6.284037e+04,21471.650000,0.000000,12.000000
25%,40.000000,4.663850e+04,70609.422500,0.000000,37.000000
50%,51.000000,5.901300e+04,88931.455000,1.000000,44.000000
75%,61.000000,7.029800e+04,107093.375000,1.000000,52.000000
max,70.000000,1.000000e+09,165482.310000,1.000000,80.000000


# 8. Ausreißer durch Median ersetzen

Alle Einkommen über 1.000.000€ Einkommen und unter 0€ Einkommen, werden durch den Median ersetzt.

In [18]:
median_einkommen = dp_1["Einkommen"].median()
median_einkommen

59013.0

In [19]:
dp_1["Einkommen"] = dp_1["Einkommen"].apply(lambda x: median_einkommen if x >=1000000 or x <= 0 else x)
dp_1.describe()

,Alter,Einkommen,Preis,Geschlecht,Zeit
count,1104.000000,1104.000000,1104.000000,1104.000000,1104.000000
mean,50.382246,58894.998188,88975.198714,0.639493,44.375906
std,11.953017,16509.072600,24464.067781,0.480365,10.848040
min,30.000000,14547.000000,21471.650000,0.000000,12.000000
25%,40.000000,46697.000000,70609.422500,0.000000,37.000000
50%,51.000000,59013.000000,88931.455000,1.000000,44.000000
75%,61.000000,70281.750000,107093.375000,1.000000,52.000000
max,70.000000,107689.000000,165482.310000,1.000000,80.000000


# 9. Typen der dp_1 und dp_2 ändern

Einkommen muss zu "Float" geändert werden (numerischer Wert)
Geschlecht muss zu "Int" geändert werden (Ganzzahlen 0 & 1)
Zeit muss zu "Float" geändert werden (numerischer Wert)

In [20]:
dp_1["Geschlecht"] = dp_1["Geschlecht"].astype("int64")
dp_1.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [21]:
dp_2["Einkommen"] = pd.to_numeric(dp_2["Einkommen"].str.replace(',','.'), errors = "coerce")
dp_2["Zeit"] = pd.to_numeric(dp_2["Zeit"].str.replace(',','.'), errors = "coerce")
dp_2.dtypes

Alter           int64
Einkommen     float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

# 10. Zusammenfügen der Datensätze

In [22]:
dp_3 = pd.concat([dp_1, dp_2],
            axis = 0,
            ignore_index = True)

In [23]:
dp_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.000000,88160.31,1,43.0,K0310
1,54,77644.000000,103145.70,1,40.0,K1042
2,55,44341.000000,80565.16,0,37.0,K0382
3,49,67271.000000,83949.89,0,42.0,K0498
4,46,49832.000000,93781.58,0,41.0,K0552
...,...,...,...,...,...,...
1990,32,34391.824965,NaN,1,48.0,K1211
1991,41,68516.495439,NaN,0,34.0,K1375
1992,65,89685.333441,NaN,0,23.0,K1965
1993,68,95714.305268,NaN,0,32.0,K1540


Einkommen sollte gerundet werden.

In [24]:
dp_3["Einkommen"] = dp_3 ["Einkommen"]. round (3)

In [25]:
dp_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.000,88160.31,1,43.0,K0310
1,54,77644.000,103145.70,1,40.0,K1042
2,55,44341.000,80565.16,0,37.0,K0382
3,49,67271.000,83949.89,0,42.0,K0498
4,46,49832.000,93781.58,0,41.0,K0552
...,...,...,...,...,...,...
1990,32,34391.825,NaN,1,48.0,K1211
1991,41,68516.495,NaN,0,34.0,K1375
1992,65,89685.333,NaN,0,23.0,K1965
1993,68,95714.305,NaN,0,32.0,K1540


# 11. Einfügen der Textdatei

In [26]:
text_datei = "data/geo.txt"
dp_text = pd.read_csv (text_datei, names = ["Platzhalter"], header = None)

In [27]:
dp_text

,Platzhalter
0,KundeNr\tNiederlassung
1,K0001\tSachsen
2,K0002\tNRW
3,K0003\tHessen
4,K0004\tBayern
...,...
1991,K1991\tHessen
1992,K1992\tBaden-Württemberg
1993,K1993\tNordrhein-Westfalen
1994,K1994\tBerlin


# 12. Überschriften ändern

In [28]:
dp_text [["KundeNR", "Niederlassung"]] = dp_text ["Platzhalter"].str.split(expand=True)

In [29]:
dp_text

,Platzhalter,KundeNR,Niederlassung
0,KundeNr\tNiederlassung,KundeNr,Niederlassung
1,K0001\tSachsen,K0001,Sachsen
2,K0002\tNRW,K0002,NRW
3,K0003\tHessen,K0003,Hessen
4,K0004\tBayern,K0004,Bayern
...,...,...,...
1991,K1991\tHessen,K1991,Hessen
1992,K1992\tBaden-Württemberg,K1992,Baden-Württemberg
1993,K1993\tNordrhein-Westfalen,K1993,Nordrhein-Westfalen
1994,K1994\tBerlin,K1994,Berlin


In [30]:
dp_text.columns

Index(['Platzhalter', 'KundeNR', 'Niederlassung'], dtype='object')

In [31]:
dp_text = dp_text.drop ("Platzhalter", axis=1)

In [32]:
dp_text.columns

Index(['KundeNR', 'Niederlassung'], dtype='object')

In [33]:
dp_3 = dp_3.copy()
dp_3 ["Niederlassung"] = None

In [34]:
dp_3

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr,Niederlassung
0,64,66894.000,88160.31,1,43.0,K0310,None
1,54,77644.000,103145.70,1,40.0,K1042,None
2,55,44341.000,80565.16,0,37.0,K0382,None
3,49,67271.000,83949.89,0,42.0,K0498,None
4,46,49832.000,93781.58,0,41.0,K0552,None
...,...,...,...,...,...,...,...
1990,32,34391.825,NaN,1,48.0,K1211,None
1991,41,68516.495,NaN,0,34.0,K1375,None
1992,65,89685.333,NaN,0,23.0,K1965,None
1993,68,95714.305,NaN,0,32.0,K1540,None


# 13. Umbenennung der Spalte KundeNr

Spalte wird umbenannt um Kompatibilitätsprobleme zu vermeiden

In [35]:
dp_3 = dp_3.rename(columns = {"KundeNr": "KundeNR"})
dp_3.dtypes

Alter              int64
Einkommen        float64
Preis            float64
Geschlecht         int64
Zeit             float64
KundeNR           object
Niederlassung     object
dtype: object

# 14. dp_3 CSV mit dp_Text-Datei mergen

Dies wird zur letzten CSV Datei.

In [36]:
merged_dp = pd.merge (dp_3, dp_text[["KundeNR", "Niederlassung"]], on = "KundeNR", how = "left")

In [37]:
merged_dp

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung_x,Niederlassung_y
0,64,66894.000,88160.31,1,43.0,K0310,None,Bayern
1,54,77644.000,103145.70,1,40.0,K1042,None,Berlin
2,55,44341.000,80565.16,0,37.0,K0382,None,Baden-Württemberg
3,49,67271.000,83949.89,0,42.0,K0498,None,Bayern
4,46,49832.000,93781.58,0,41.0,K0552,None,Hessen
...,...,...,...,...,...,...,...,...
1990,32,34391.825,NaN,1,48.0,K1211,None,Hessen
1991,41,68516.495,NaN,0,34.0,K1375,None,Sachsen
1992,65,89685.333,NaN,0,23.0,K1965,None,Bayern
1993,68,95714.305,NaN,0,32.0,K1540,None,Bayern


# 15. Spalten ändern 

Spalte Niederlassung_x muss gelöscht werden, da sie überflüssig ist und Niederlassung_y muss umbenannt werden

In [38]:
merged_dp = merged_dp.rename(columns={"Niederlassung_y": "Niederlassung"})
merged_dp = merged_dp.drop("Niederlassung_x", axis = 1)

In [39]:
merged_dp

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNR,Niederlassung
0,64,66894.000,88160.31,1,43.0,K0310,Bayern
1,54,77644.000,103145.70,1,40.0,K1042,Berlin
2,55,44341.000,80565.16,0,37.0,K0382,Baden-Württemberg
3,49,67271.000,83949.89,0,42.0,K0498,Bayern
4,46,49832.000,93781.58,0,41.0,K0552,Hessen
...,...,...,...,...,...,...,...
1990,32,34391.825,NaN,1,48.0,K1211,Hessen
1991,41,68516.495,NaN,0,34.0,K1375,Sachsen
1992,65,89685.333,NaN,0,23.0,K1965,Bayern
1993,68,95714.305,NaN,0,32.0,K1540,Bayern


# 16. Niederlassungen anzeigen lassen

Mit dem Befehl .unique() werden alle einzigartigen Werte aus der Spalte.

In [40]:
merged_dp["Niederlassung"].unique()

array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
       'Sachsen', 'Nordrhein-Westfalen', 'BERLIN', 'Niedersachsen',
       'Hamburg', 'Brandenburg', 'Berlin-Mitte', 'Düsseldorf', 'NRW',
       'Berlin-Charlottenburg'], dtype=object)

# 17. Niederlassungen den Bundesländern zuordnen 

Abkürzungen und Doppelungen innerhalb eines Bundeslandes werden zusammengeführt.

In [41]:
merged_dp["Niederlassung"] = merged_dp ["Niederlassung"].replace("Düsseldorf", "Nordrhein-Westfalen")
merged_dp["Niederlassung"] = merged_dp ["Niederlassung"].replace("BERLIN", "Berlin")
merged_dp["Niederlassung"] = merged_dp ["Niederlassung"].replace("Berlin-Charlottenburg", "Berlin")
merged_dp["Niederlassung"] = merged_dp ["Niederlassung"].replace("Berlin-Mitte", "Berlin")
merged_dp["Niederlassung"] = merged_dp ["Niederlassung"].replace("NRW", "Nordrhein-Westfalen")

In [42]:
merged_dp["Niederlassung"].unique()

array(['Bayern', 'Berlin', 'Baden-Württemberg', 'Hessen', 'Thüringen',
       'Sachsen', 'Nordrhein-Westfalen', 'Niedersachsen', 'Hamburg',
       'Brandenburg'], dtype=object)

# 18. Eine neue Spalte erstellen namens Kunde und diese umpositionieren

In [45]:
merged_dp["Kunde"] = merged_dp ["Preis"]. apply(lambda x: 1 if pd.notna(x) else 0)

In [46]:
merged_dp.insert(0, "Kunde", merged_dp.pop("Kunde"))

# 19. CSV Speichern

In [47]:
merged_dp.to_csv("data/dp_final.csv", index = False)

In [49]:
dp_1.to_csv("data/dp_1.csv", index = False)

In [50]:
dp_2.to_csv("data/dp_2.csv", index = False)

In [51]:
dp_3.to_csv("data/dp_3.csv", index = False)